<a href="https://colab.research.google.com/github/khadijaraza/spisproject/blob/main/data_refinement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lightkurve

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.6/199.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 110.3 MB/s eta 0:00:00
  Created wheel for fbpca: filename=fbpca-1.0-py3-none-any.whl size=11373 sha256=12dc691447f2b9b78cbacf2ca0dad264e83fd628515db3c2c3815981b83ab912
  Stored in directory: /root/.cache/pip/wheels/04/15/cd/2f622795b09e83471a3be5d2581cd9cf96a6ec7aa78e8deffe
  Created wheel for memoizatio

In [ ]:
!pip install astroquery

In [ ]:
import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
from astroquery.mast import Catalogs
#from astroquery.mast import DiscoveryPortal # Added DiscoveryPortal import
import pandas as pd
from tqdm import tqdm # For a helpful progress bar
from collections import defaultdict, Counter
import pickle
import warnings

/usr/local/lib/python3.12/dist-packages/lightkurve/prf/__init__.py:7: UserWarning: Warning: the tpfmodel submodule is not available without oktopus installed, which requires a current version of autograd. See #1452 for details.
  warnings.warn(


In [4]:
warnings.filterwarnings('ignore', message='.*has a unit but is kept as a Column.*')
print("Downloading confirmed planet data from NASA Exoplanet Archive...")
try:
    confirmed_planets_catalog = NasaExoplanetArchive.query_criteria(
        table="pscomppars", select="tic_id, pl_name"
    )
    confirmed_planets_df = confirmed_planets_catalog.to_pandas()
    if 'tic_id' in confirmed_planets_df.columns and not confirmed_planets_df['tic_id'].isnull().all():
        planet_tics = [f"TIC {tic_id}" for tic_id in confirmed_planets_df['tic_id'].dropna().astype(str).str.replace('TIC ', '').unique()]
        print(f"Found {len(planet_tics)} unique TIC IDs with confirmed planets.")
    else:
        planet_tics = []
except Exception as e:
    print(f"Could not query NASA Exoplanet Archive for confirmed planets: {e}")
    planet_tics = []

print("\nDownloading false positive data from NASA Exoplanet Archive (TOI Table)...")
try:
    toi_catalog = NasaExoplanetArchive.query_criteria(
        table="toi", select="tid, tfopwg_disp"
    )
    toi_df = toi_catalog.to_pandas()
    fp_mask = toi_df['tfopwg_disp'] == 'FP'
    fp_df = toi_df[fp_mask]
    fp_tics = [f"TIC {int(tid)}" for tid in fp_df['tid'].dropna().unique()]
    print(f"Found {len(fp_tics)} unique TIC IDs classified as False Positives (stars).")
except Exception as e:
    print(f"Could not query TOI Catalog for false positives: {e}")
    fp_tics = []


# --- Step 3: Download Planet Light Curves ---
print("\n--- Downloading Planet Light Curves (One by One) ---")
list_of_planet_collections = []
if planet_tics:
    tics_to_download = planet_tics[:500]
    print(f"Downloading all TESS light curves for {len(tics_to_download)} hosts...")

    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        for tic in tqdm(tics_to_download):
            try:
                search_result = lk.search_lightcurve(target=tic, mission="TESS")
                spoc_result = search_result[search_result.author == "TESS-SPOC"]
                if len(spoc_result) > 0:
                    lc_collection_for_tic = spoc_result.download_all()
                    list_of_planet_collections.append(lc_collection_for_tic)
            except Exception:
                continue

    print(f"Successfully created a list of {len(list_of_planet_collections)} collections for planets.")
else:
    print("No planet TICs to download.")


# --- Step 4: Download False Positive Light Curves ---
print("\n--- Downloading False Positive Light Curves (One by One) ---")
list_of_fp_collections = []
if fp_tics:
    tics_to_download = fp_tics[:500]
    print(f"Downloading all TESS light curves for {len(tics_to_download)} false positives...")

    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        for tic in tqdm(tics_to_download):
            try:
                search_result = lk.search_lightcurve(target=tic, mission="TESS")
                spoc_result = search_result[search_result.author == "TESS-SPOC"]
                if len(spoc_result) > 0:
                    lc_collection_for_tic = spoc_result.download_all()
                    list_of_fp_collections.append(lc_collection_for_tic)
            except Exception:
                continue

    print(f"Successfully created a list of {len(list_of_fp_collections)} collections for false positives.")
else:
    print("No false positive TICs to download.")

Found 4223 unique TIC IDs with confirmed planets.

Found 1186 unique TIC IDs classified as False Positives (stars).

--- Downloading Planet Light Curves (One by One) ---


 13%|█▎        | 66/500 [07:05<1:02:44,  8.67s/it]Could not resolve TIC  to a sky position.
ERROR:lightkurve.search:Could not resolve TIC  to a sky position.
 14%|█▎        | 68/500 [07:07<34:53,  4.85s/it]No data found for target "TIC 459558974".
ERROR:lightkurve.search:No data found for target "TIC 459558974".
 17%|█▋        | 86/500 [08:10<21:25,  3.11s/it]No data found for target "TIC 281520119".
ERROR:lightkurve.search:No data found for target "TIC 281520119".
 18%|█▊        | 92/500 [08:52<46:46,  6.88s/it]No data found for target "TIC 467929202".
ERROR:lightkurve.search:No data found for target "TIC 467929202".
 25%|██▍       | 123/500 [11:15<10:35,  1.69s/it]No data found for target "TIC 323319664".
ERROR:lightkurve.search:No data found for target "TIC 323319664".
 30%|███       | 151/500 [13:00<13:39,  2.35s/it]No data found for target "TIC 1685332572".
ERROR:lightkurve.search:No data found for target "TIC 1685332572".
 35%|███▍      | 173/500 [15:08<21:32,  3.95s/it]No data f

Successfully created a list of 408 collections for planets.

--- Downloading False Positive Light Curves (One by One) ---


100%|██████████| 500/500 [48:58<00:00,  5.88s/it]

Successfully created a list of 474 collections for false positives.


In [5]:
#outdated data scrapping
'''
try:
    print("Searching for data in TESS...")
    search_result = lk.search_lightcurve(SkyCoord.from_name("Galactic Center"), mission='TESS', radius=0.25*u.degree, exposure = 600.0)

    #print("Filtering for 600s exposure time...")
    #filtered_results = search_result[search_result.exptime.value == 600]
    filtered_results = search_result

    print(f"Found {len(filtered_results)} light curves. Downloading...")
    lc_collection = filtered_results.download_all()
    print("Download complete.")

except Exception as e:
    print(f"An error occurred during download: {e}")
    lc_collection = None
'''

'\ntry:\n    print("Searching for data in TESS...")\n    search_result = lk.search_lightcurve(SkyCoord.from_name("Galactic Center"), mission=\'TESS\', radius=0.25*u.degree, exposure = 600.0)\n\n    #print("Filtering for 600s exposure time...")\n    #filtered_results = search_result[search_result.exptime.value == 600]\n    filtered_results = search_result\n\n    print(f"Found {len(filtered_results)} light curves. Downloading...")\n    lc_collection = filtered_results.download_all()\n    print("Download complete.")\n\nexcept Exception as e:\n    print(f"An error occurred during download: {e}")\n    lc_collection = None\n'

In [6]:
def find_period_from_sectors(lc_collection, min_power=10, max_search_points=50000):
    if not lc_collection:
        return None

    results = []
    # Define the transit durations to search for (~1 to 12 hours)
    duration_grid = np.linspace(0.04, 0.5, 20)

    for lc_sector in lc_collection:
        try:
            bls = lc_sector.remove_nans().to_periodogram(
                method='bls',
                duration=duration_grid,
                # --- FINAL FIX: Add a minimum_period constraint ---
                # This prevents the error for short-period objects.
                minimum_period=np.max(duration_grid) + 0.01
            )
            power = np.max(bls.power.value)

            if power > min_power:
                period = bls.period_at_max_power
                duration = bls.duration_at_max_power
                transit_time = bls.transit_time_at_max_power
                # Store the results with the power as a score
                results.append({'period': period, 'duration': duration,
                                'transit_time': transit_time, 'power': power})
        except Exception:
            continue

    if results:
        # Return the dictionary of parameters from the search with the highest power
        best_result = max(results, key=lambda x: x['power'])
        return best_result
    else:
        return None

In [7]:
def condenseData(list_of_collections):

    if not list_of_collections:
        return []

    stitched_lightcurves = []
    # Use tqdm for a progress bar
    for lc_collection in tqdm(list_of_collections, desc="Stitching collections"):
        params = find_period_from_sectors(lc_collection)
        try:
            # Stitch all light curves within this one collection
            stitched_lc = lc_collection.stitch()
            stitched_lightcurves.append((stitched_lc, params))
        except Exception as e:
            # Gracefully skip any collections that fail to stitch
            target_id = lc_collection[0].TARGETID if lc_collection else 'Unknown'
            print(f"\nWarning: Could not stitch collection for TIC {target_id}. Reason: {e}")
            continue

    print(f"Successfully stitched {len(stitched_lightcurves)} light curves.")
    return stitched_lightcurves

In [8]:
def preprocess_lightcurve(lc_tuple):
    stitched_lc, params = lc_tuple
    if not params:
        return None
    try:
        period = params['period']
        duration = params['duration']
        transit_time = params['transit_time']

        # Dynamically set the window size for flattening
        window = int(duration.value * 5)
        if window % 2 == 0: window += 1
        if window < 21: window = 21

        # Flatten, fold, and prepare the final data product
        flat_lc = stitched_lc.flatten(window_length=window)
        folded_lc = flat_lc.fold(period=period, epoch_time=transit_time)
        zoomed_lc = folded_lc[(folded_lc.time.value > -0.5) & (folded_lc.time.value < 0.5)]
        binned_lc = zoomed_lc.bin(time_bin_size=0.01)

        # Final safety check and manual normalization
        median_flux = np.nanmedian(binned_lc.flux.value)
        if np.isclose(median_flux, 0): return None
        processed_lc = binned_lc / median_flux

        return processed_lc
    except Exception as e:
        target_id = stitched_lc.TARGETID if hasattr(stitched_lc, 'TARGETID') else 'Unknown'
        print(f"Could not process light curve for target {target_id}. Reason: {e}")
        return None

In [9]:
stitched_lightcurves_planets = condenseData(list_of_planet_collections)
stitched_lightcurves_fps = condenseData(list_of_fp_collections)

Stitching collections: 100%|██████████| 408/408 [11:31<00:00,  1.69s/it]


Successfully stitched 408 light curves.


Stitching collections: 100%|██████████| 474/474 [19:58<00:00,  2.53s/it]

Successfully stitched 474 light curves.


In [ ]:
processed_lcs_planets = []
processed_lcs_fps = []
for lc in stitched_lightcurves_planets:
    processed_lcs_planets.append(preprocess_lightcurve(lc))
for lc in stitched_lightcurves_fps:
    processed_lcs_fps.append(preprocess_lightcurve(lc))



In [ ]:
def sufficient_snr(processed_lc, min_snr=1):
    out_of_dip_mask = np.abs(processed_lc.time.value) > 0.1
    noise = np.std(processed_lc.flux.value[out_of_dip_mask])

    # The dip depth is the minimum flux value (since the baseline is at 1.0)
    dip_depth = 1.0 - np.min(processed_lc.flux.value)

    # Calculate the Signal-to-Noise Ratio
    snr = dip_depth / noise

    return snr >= min_snr

In [ ]:
def prepareDataset (planets, fps):
  dataset = []
  for lc in planets:
    if sufficient_snr(lc):
      dataset.append((lc, True))
  for lc in fps:
    if sufficient_snr(lc):
      dataset.append((lc, False))
  return dataset


In [ ]:
output_filename = 'my_dataset.pkl'
full_dataset = prepareDataset(processed_lcs_planets, processed_lcs_fps)
# Use 'wb' to write the file in binary mode
with open(output_filename, 'wb') as f:
    pickle.dump(full_dataset, f)

print(f"Dataset with {len(full_dataset)} items was successfully saved to '{output_filename}'")

In [ ]:
for lc in processed_lcs_planets:
    if sufficient_snr(lc):
      lc.plot(title="Planet Candidate") # Add a title for planet light curves
      #plt.close() # Close the plot to avoid the warning


for lc in processed_lcs_fps:
    if sufficient_snr(lc):
      lc.plot(title="False Positive (Star)") # Add a title for false positive light curves
      #plt.close() # Close the plot to avoid the warning